In [1]:
!pip install tensorflow
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
df = pd.read_csv('/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
df.TotalCharges = pd.to_numeric(df.TotalCharges, errors='coerce')

In [4]:
df.drop('customerID', axis = 'columns', inplace = True)

In [5]:
df.replace('No phone service', 'No', inplace = True)
df.replace('No internet service', 'No', inplace = True)

In [6]:
df.replace({'Female': 1, 'Male': 0}, inplace = True)

In [7]:
df.replace({'Yes': 1, 'No': 0}, inplace = True)

In [8]:
df.InternetService = [i if(i != 0) else 'No' for i in df.InternetService]

In [9]:
df.InternetService.value_counts()

Fiber optic    3096
DSL            2421
No             1526
Name: InternetService, dtype: int64

In [10]:
col_for_encoding = ['InternetService', 'Contract', 'PaymentMethod']
final_df = pd.get_dummies(data = df, columns=col_for_encoding)
final_df.shape

(7043, 27)

In [11]:
df.shape

(7043, 20)

In [12]:
final_df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,InternetService_DSL,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1,0,1,0,1,0,0,0,1,0,...,1,0,0,1,0,0,0,0,1,0
1,0,0,0,0,34,1,0,1,0,1,...,1,0,0,0,1,0,0,0,0,1
2,0,0,0,0,2,1,0,1,1,0,...,1,0,0,1,0,0,0,0,0,1
3,0,0,0,0,45,0,0,1,0,1,...,1,0,0,0,1,0,1,0,0,0
4,1,0,0,0,2,1,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0


In [13]:
final_df.isna().any()

gender                                     False
SeniorCitizen                              False
Partner                                    False
Dependents                                 False
tenure                                     False
PhoneService                               False
MultipleLines                              False
OnlineSecurity                             False
OnlineBackup                               False
DeviceProtection                           False
TechSupport                                False
StreamingTV                                False
StreamingMovies                            False
PaperlessBilling                           False
MonthlyCharges                             False
TotalCharges                                True
Churn                                      False
InternetService_DSL                        False
InternetService_Fiber optic                False
InternetService_No                         False
Contract_Month-to-mo

In [14]:
df.dropna(inplace = True)

In [15]:
final_df.dropna(inplace = True)

In [16]:
final_df.shape

(7032, 27)

In [17]:
x = final_df.drop('Churn', axis = 'columns')
y = final_df.Churn

In [18]:
col_to_scale = ['tenure', 'MonthlyCharges', 'TotalCharges']

In [19]:
from sklearn.preprocessing import MinMaxScaler

In [20]:
scaler = MinMaxScaler()
x[col_to_scale] = scaler.fit_transform(x[col_to_scale])

In [21]:
x.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,InternetService_DSL,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1,0,1,0,0.000000,0,0,0,1,0,...,1,0,0,1,0,0,0,0,1,0
1,0,0,0,0,0.464789,1,0,1,0,1,...,1,0,0,0,1,0,0,0,0,1
2,0,0,0,0,0.014085,1,0,1,1,0,...,1,0,0,1,0,0,0,0,0,1
3,0,0,0,0,0.619718,0,0,1,0,1,...,1,0,0,0,1,0,1,0,0,0
4,1,0,0,0,0.014085,1,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [23]:
def ANN(x_train, y_train, x_test, y_test):
    model = keras.Sequential([
        keras.layers.Dense(20, input_shape = (x_train.shape[1],), activation = 'relu'),
        keras.layers.Dense(10, activation = 'relu'),
        keras.layers.Dense(1, activation = 'sigmoid')
    ])

    model.compile(
        optimizer = 'adam',
        loss = 'binary_crossentropy',
        metrics = ['accuracy']
    )

    model.fit(x_train, y_train, epochs = 100)
    print(model.evaluate(x_test, y_test))
    y_pred = np.round(model.predict(x_test))

    cr = classification_report(y_test, y_pred)
    print('Classification report: \n', cr)

    return y_pred

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=5)
ANN(x_train, y_train, x_test, y_test)

Epoch 1/100
176/176 [==============================] - 6s 3ms/step - loss: 0.5262 - accuracy: 0.7269
Epoch 2/100
176/176 [==============================] - 1s 4ms/step - loss: 0.4333 - accuracy: 0.7902
Epoch 3/100
176/176 [==============================] - 1s 6ms/step - loss: 0.4206 - accuracy: 0.7950
Epoch 4/100
176/176 [==============================] - 1s 4ms/step - loss: 0.4156 - accuracy: 0.8028
Epoch 5/100
176/176 [==============================] - 1s 4ms/step - loss: 0.4134 - accuracy: 0.8048
Epoch 6/100
176/176 [==============================] - 1s 3ms/step - loss: 0.4106 - accuracy: 0.8050
Epoch 7/100
176/176 [==============================] - 1s 3ms/step - loss: 0.4087 - accuracy: 0.8078
Epoch 8/100
176/176 [==============================] - 1s 4ms/step - loss: 0.4075 - accuracy: 0.8085
Epoch 9/100
176/176 [==============================] - 1s 4ms/step - loss: 0.4064 - accuracy: 0.8091
Epoch 10/100
176/176 [==============================] - 1s 4ms/step - loss: 0.4055 - accura

array([[0.],
       [1.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

**Handling Imbalance Dataset**
1. Under sampling

In [25]:
df1 = x.copy()
df1['Churn'] = y

In [26]:
df = df1
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
0,1,0,1,0,0.000000,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
1,0,0,0,0,0.464789,1,0,1,0,1,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,0,0.014085,1,0,1,1,0,...,0,0,1,0,0,0,0,0,1,1
3,0,0,0,0,0.619718,0,0,1,0,1,...,0,0,0,1,0,1,0,0,0,0
4,1,0,0,0,0.014085,1,0,0,0,0,...,1,0,1,0,0,0,0,1,0,1


In [27]:
df1.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
0,1,0,1,0,0.000000,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
1,0,0,0,0,0.464789,1,0,1,0,1,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,0,0.014085,1,0,1,1,0,...,0,0,1,0,0,0,0,0,1,1
3,0,0,0,0,0.619718,0,0,1,0,1,...,0,0,0,1,0,1,0,0,0,0
4,1,0,0,0,0.014085,1,0,0,0,0,...,1,0,1,0,0,0,0,1,0,1


In [28]:
df1_class_0 = df1[df1.Churn == 0]
df1_class_1 = df1[df1.Churn == 1]

df1_class_0.shape, df1_class_1.shape, df.shape

((5163, 27), (1869, 27), (7032, 27))

In [29]:
df1_class_0_under = df1_class_0.sample(len(df1_class_1))
df1_class_0_under.shape

(1869, 27)

In [30]:
df1_merged = pd.concat([df1_class_0_under, df1_class_1], axis = 'rows')
df1_merged.shape

(3738, 27)

In [31]:
df1_merged.Churn.value_counts()

0    1869
1    1869
Name: Churn, dtype: int64

In [32]:
x = df1_merged.drop('Churn', axis = 'columns')
y = df1_merged.Churn

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1, stratify=y)

In [33]:
y_test.value_counts()

0    374
1    374
Name: Churn, dtype: int64

In [34]:
y_train.value_counts()

1    1495
0    1495
Name: Churn, dtype: int64

In [35]:
y_pred1 = ANN(x_train, y_train, x_test, y_test)

Epoch 1/100
94/94 [==============================] - 2s 3ms/step - loss: 0.6421 - accuracy: 0.6492
Epoch 2/100
94/94 [==============================] - 0s 3ms/step - loss: 0.5341 - accuracy: 0.7388
Epoch 3/100
94/94 [==============================] - 0s 4ms/step - loss: 0.4998 - accuracy: 0.7589
Epoch 4/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4930 - accuracy: 0.7612
Epoch 5/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4888 - accuracy: 0.7622
Epoch 6/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4850 - accuracy: 0.7642
Epoch 7/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4837 - accuracy: 0.7686
Epoch 8/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4824 - accuracy: 0.7679
Epoch 9/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4801 - accuracy: 0.7699
Epoch 10/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4787 - accuracy: 0.7686
Epoch 11/

2. Over sampling of minority class

In [36]:
df2 = df
df2.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
0,1,0,1,0,0.000000,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
1,0,0,0,0,0.464789,1,0,1,0,1,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,0,0.014085,1,0,1,1,0,...,0,0,1,0,0,0,0,0,1,1
3,0,0,0,0,0.619718,0,0,1,0,1,...,0,0,0,1,0,1,0,0,0,0
4,1,0,0,0,0.014085,1,0,0,0,0,...,1,0,1,0,0,0,0,1,0,1


In [37]:
df2_class_0 = df2[df2.Churn == 0]
df2_class_1 = df2[df2.Churn == 1]

df2_class_0.shape, df2_class_1.shape

((5163, 27), (1869, 27))

In [38]:
df2_class_1_over = df2_class_1.sample(len(df2_class_0), replace = True)
df2_class_1_over.shape

(5163, 27)

In [39]:
df2_merged = pd.concat([df2_class_0, df2_class_1_over], axis = 'rows')
df2_merged.shape

(10326, 27)

In [40]:
x = df2_merged.drop('Churn', axis = 'columns')
y = df2_merged.Churn

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1, stratify=y)

In [41]:
y_train.value_counts()

0    4130
1    4130
Name: Churn, dtype: int64

In [42]:
y_test.value_counts()

0    1033
1    1033
Name: Churn, dtype: int64

In [43]:
ANN(x_train, y_train, x_test, y_test)

Epoch 1/100
259/259 [==============================] - 2s 3ms/step - loss: 0.5848 - accuracy: 0.6942
Epoch 2/100
259/259 [==============================] - 1s 4ms/step - loss: 0.4916 - accuracy: 0.7673
Epoch 3/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4825 - accuracy: 0.7695
Epoch 4/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4788 - accuracy: 0.7724
Epoch 5/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4749 - accuracy: 0.7711
Epoch 6/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4725 - accuracy: 0.7714
Epoch 7/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4687 - accuracy: 0.7772
Epoch 8/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4667 - accuracy: 0.7774
Epoch 9/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4641 - accuracy: 0.7783
Epoch 10/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4633 - accura

array([[1.],
       [0.],
       [0.],
       ...,
       [1.],
       [0.],
       [0.]], dtype=float32)

3. SMOTE method

In [44]:
!pip install imbalanced-learn

In [45]:
df3 = df.copy()
df3.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
0,1,0,1,0,0.000000,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
1,0,0,0,0,0.464789,1,0,1,0,1,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,0,0.014085,1,0,1,1,0,...,0,0,1,0,0,0,0,0,1,1
3,0,0,0,0,0.619718,0,0,1,0,1,...,0,0,0,1,0,1,0,0,0,0
4,1,0,0,0,0.014085,1,0,0,0,0,...,1,0,1,0,0,0,0,1,0,1


In [46]:
x = df3.drop('Churn', axis = 'columns')
y = df3.Churn

In [47]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
x_sm, y_sm = smote.fit_resample(x, y)

In [48]:
x_sm.shape

(10326, 26)

In [49]:
y_sm.shape

(10326,)

In [50]:
x_sm.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,InternetService_DSL,InternetService_Fiber optic,InternetService_No,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1,0,1,0,0.000000,0,0,0,1,0,...,1,0,0,1,0,0,0,0,1,0
1,0,0,0,0,0.464789,1,0,1,0,1,...,1,0,0,0,1,0,0,0,0,1
2,0,0,0,0,0.014085,1,0,1,1,0,...,1,0,0,1,0,0,0,0,0,1
3,0,0,0,0,0.619718,0,0,1,0,1,...,1,0,0,0,1,0,1,0,0,0
4,1,0,0,0,0.014085,1,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0


In [51]:
y_sm[:5]

0    0
1    0
2    1
3    0
4    1
Name: Churn, dtype: int64

In [52]:
x_train, x_test, y_train, y_test = train_test_split(x_sm, y_sm, test_size=0.2, random_state=1, stratify = y_sm)

In [53]:
y_train.value_counts()

0    4130
1    4130
Name: Churn, dtype: int64

In [54]:
y_test.value_counts()

0    1033
1    1033
Name: Churn, dtype: int64

In [55]:
ANN(x_train, y_train, x_test, y_test)

Epoch 1/100
259/259 [==============================] - 2s 3ms/step - loss: 0.5365 - accuracy: 0.7360
Epoch 2/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4713 - accuracy: 0.7774
Epoch 3/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4627 - accuracy: 0.7823
Epoch 4/100
259/259 [==============================] - 1s 5ms/step - loss: 0.4578 - accuracy: 0.7828
Epoch 5/100
259/259 [==============================] - 1s 4ms/step - loss: 0.4549 - accuracy: 0.7840
Epoch 6/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4497 - accuracy: 0.7878
Epoch 7/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4471 - accuracy: 0.7856
Epoch 8/100
259/259 [==============================] - 1s 4ms/step - loss: 0.4446 - accuracy: 0.7915
Epoch 9/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4407 - accuracy: 0.7932
Epoch 10/100
259/259 [==============================] - 1s 3ms/step - loss: 0.4372 - accura

array([[1.],
       [0.],
       [0.],
       ...,
       [1.],
       [0.],
       [0.]], dtype=float32)

4. Ensemble method

In [56]:
df4 = df.copy()

In [57]:
df4.shape

(7032, 27)

In [58]:
df4['Churn'].value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [59]:
df4_class_0 = df4[df4.Churn == 0]
df4_class_1 = df4[df4.Churn == 1]

df4_class_0.shape, df4_class_1.shape

((5163, 27), (1869, 27))

In [60]:
df4_class_0_b1 = df4_class_0[:len(df4_class_1)]
df4_class_0_b1.shape

(1869, 27)

In [61]:
df4_class_0_b2 = df4_class_0[len(df4_class_1) : 2 * len(df4_class_1)]
df4_class_0_b2.shape

(1869, 27)

In [62]:
df4_class_0_b3 = df4_class_0[2 * len(df4_class_1) :]
df4_class_0_b3.shape

(1425, 27)

In [63]:
df4_merged_1 = pd.concat([df4_class_0_b1, df4_class_1], axis = 'rows') 
df4_merged_2 = pd.concat([df4_class_0_b2, df4_class_1], axis = 'rows')
df4_merged_3 = pd.concat([df4_class_0_b3, df4_class_1], axis = 'rows')

In [64]:
df4_merged_1.shape, df4_merged_2.shape, df4_merged_3.shape

((3738, 27), (3738, 27), (3294, 27))

In [65]:
def get_train_test_data(df):
    x = df.drop('Churn', axis = 'columns')
    y = df.Churn
    
    return train_test_split(x, y, test_size=0.2, random_state=1, stratify = y)

In [66]:
x_train, x_test, y_train, y_test = get_train_test_data(df4_merged_1)
y_pred1 = ANN(x_train, y_train, x_test, y_test)

Epoch 1/100
94/94 [==============================] - 2s 3ms/step - loss: 0.6396 - accuracy: 0.6656
Epoch 2/100
94/94 [==============================] - 0s 3ms/step - loss: 0.5386 - accuracy: 0.7395
Epoch 3/100
94/94 [==============================] - 0s 4ms/step - loss: 0.5091 - accuracy: 0.7502
Epoch 4/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4996 - accuracy: 0.7562
Epoch 5/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4951 - accuracy: 0.7552
Epoch 6/100
94/94 [==============================] - 0s 4ms/step - loss: 0.4910 - accuracy: 0.7579
Epoch 7/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4881 - accuracy: 0.7619
Epoch 8/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4862 - accuracy: 0.7619
Epoch 9/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4840 - accuracy: 0.7662
Epoch 10/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4825 - accuracy: 0.7662
Epoch 11/

In [67]:
y_test_final = y_test

In [68]:
x_train, x_test, y_train, y_test = get_train_test_data(df4_merged_2)
y_pred2 = ANN(x_train, y_train, x_test, y_test)
y_test_final.append(y_test)

Epoch 1/100
94/94 [==============================] - 2s 3ms/step - loss: 0.6493 - accuracy: 0.6381
Epoch 2/100
94/94 [==============================] - 0s 3ms/step - loss: 0.5602 - accuracy: 0.7478
Epoch 3/100
94/94 [==============================] - 0s 3ms/step - loss: 0.5177 - accuracy: 0.7512
Epoch 4/100
94/94 [==============================] - 0s 3ms/step - loss: 0.5022 - accuracy: 0.7559
Epoch 5/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4970 - accuracy: 0.7605
Epoch 6/100
94/94 [==============================] - 0s 4ms/step - loss: 0.4939 - accuracy: 0.7548
Epoch 7/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4906 - accuracy: 0.7612
Epoch 8/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4891 - accuracy: 0.7639
Epoch 9/100
94/94 [==============================] - 0s 3ms/step - loss: 0.4875 - accuracy: 0.7615
Epoch 10/100
94/94 [==============================] - 0s 4ms/step - loss: 0.4855 - accuracy: 0.7666
Epoch 11/

/tmp/ipykernel_28/712394417.py:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_test_final.append(y_test)


2454    0
2320    0
4027    1
2525    0
1827    0
       ..
3350    1
4377    0
2638    1
4886    0
5901    1
Name: Churn, Length: 1496, dtype: int64

In [69]:
x_train, x_test, y_train, y_test = get_train_test_data(df4_merged_3)
y_pred3 = ANN(x_train, y_train, x_test, y_test)
y_test_final.append(y_test)

Epoch 1/100
83/83 [==============================] - 1s 3ms/step - loss: 0.5946 - accuracy: 0.7222
Epoch 2/100
83/83 [==============================] - 0s 3ms/step - loss: 0.5093 - accuracy: 0.7651
Epoch 3/100
83/83 [==============================] - 0s 3ms/step - loss: 0.4851 - accuracy: 0.7727
Epoch 4/100
83/83 [==============================] - 0s 4ms/step - loss: 0.4789 - accuracy: 0.7776
Epoch 5/100
83/83 [==============================] - 0s 3ms/step - loss: 0.4727 - accuracy: 0.7795
Epoch 6/100
83/83 [==============================] - 0s 3ms/step - loss: 0.4720 - accuracy: 0.7791
Epoch 7/100
83/83 [==============================] - 0s 3ms/step - loss: 0.4666 - accuracy: 0.7829
Epoch 8/100
83/83 [==============================] - 0s 3ms/step - loss: 0.4643 - accuracy: 0.7882
Epoch 9/100
83/83 [==============================] - 0s 3ms/step - loss: 0.4623 - accuracy: 0.7856
Epoch 10/100
83/83 [==============================] - 0s 3ms/step - loss: 0.4596 - accuracy: 0.7879
Epoch 11/

/tmp/ipykernel_28/838037603.py:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_test_final.append(y_test)


2454    0
2320    0
4027    1
2525    0
1827    0
       ..
2191    1
6292    0
6936    1
948     1
5630    1
Name: Churn, Length: 1407, dtype: int64

In [70]:
y_pred1.shape

(748, 1)

In [71]:
def get_combined_result(y1, y2, y3):
    y_comb = []
    for i in range(len(y1)):
        if(i < len(y3)):
            sum = y1[i] + y2[i] + y3[i]
        else: sum = y1[i] + y2[i]
        if(sum > 1):
            y_comb.append(1)
        else: 
            y_comb.append(0)
    return y_comb

In [72]:
y_pred_final = get_combined_result(y_pred1, y_pred2, y_pred3)

In [73]:
len(y_test_final)

748

In [74]:
y_test_final

2454    0
2320    0
4027    1
2525    0
1827    0
       ..
3350    1
1842    0
2638    1
2353    0
5901    1
Name: Churn, Length: 748, dtype: int64

In [75]:
len(y_pred_final)

748

In [76]:
print(classification_report(y_test_final, y_pred_final))

              precision    recall  f1-score   support

           0       0.79      0.68      0.73       374
           1       0.72      0.81      0.76       374

    accuracy                           0.75       748
   macro avg       0.75      0.75      0.75       748
weighted avg       0.75      0.75      0.75       748

